## Warm Start

Warm start method ensures that model does not start from the 0 and the training pipeline on the first step is already populated by progressing trainees.

In [1]:
import pandas as pd

In [2]:
import sys
sys.path.append("../") # go to parent dir

In [3]:
print(sys.path)

['C:\\Users\\a.poghosyan\\Documents\\training_simulation\\training_batch_run_workflow\\training_simulator\\notebooks', 'C:\\Users\\a.poghosyan\\AppData\\Local\\Continuum\\anaconda3\\python312.zip', 'C:\\Users\\a.poghosyan\\AppData\\Local\\Continuum\\anaconda3\\DLLs', 'C:\\Users\\a.poghosyan\\AppData\\Local\\Continuum\\anaconda3\\Lib', 'C:\\Users\\a.poghosyan\\AppData\\Local\\Continuum\\anaconda3', '', 'C:\\Users\\a.poghosyan\\AppData\\Local\\Continuum\\anaconda3\\Lib\\site-packages', 'C:\\Users\\a.poghosyan\\AppData\\Local\\Continuum\\anaconda3\\Lib\\site-packages\\win32', 'C:\\Users\\a.poghosyan\\AppData\\Local\\Continuum\\anaconda3\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\a.poghosyan\\AppData\\Local\\Continuum\\anaconda3\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\a.poghosyan\\AppData\\Local\\Continuum\\anaconda3\\Lib\\site-packages\\setuptools\\_vendor', '../']


In [4]:
DEFAULT_PARAMETERS = {"version": "2.1.3",
    "simulation": {"steps": 120, "start_month": 4, "iterations": 5},
    "streaming":1,
    "init_trainees": {
        "course1": {"progressing": 0, "hold": 0},
        "course2": {"progressing": 0, "hold": 0},
        "course3": {"progressing": 0, "hold": 0},
        "course4": {"progressing": 0, "hold": 0},
        "course5": {"progressing": 0, "hold": 0},
        "course6": {"progressing": 0, "hold": 0},
        "course7": {"progressing": 0, "hold": 0},

    },

    "pipeline": {
        "init": {"new_trainees": 10, "input_rate": 1, "time_hold": 120},
        "course1": {
            "drop_out_progressing": 0.12,
            "drop_out_hold": 0,
            "capacity_progressing": 21,
            "time_progressing": 2,
            "time_hold": 120,
        },
        "course2": {
            "drop_out_progressing": 0.05,
            "drop_out_stream": 0.6,
            "drop_out_hold": 0,
            "capacity_progressing": 11,
            "time_progressing": 6,
            "time_hold": 120,
        },
        "course3": {
            "drop_out_progressing": 0.0,
            "drop_out_hold": 0.0,
            "capacity_progressing": 10,
            "time_progressing": 2,
            "time_hold": 120,
        },
        "course4": {
            "drop_out_progressing": 0.05,
            "drop_out_hold": 0,
            "time_progressing": 13,
            "capacity_progressing": 4,
            "pathway_complete": "training_pathway1_complete",
            "time_hold": 120,
        },
        "course5": {
            "drop_out_progressing": 0.2,
            "drop_out_hold": 0,
            "capacity_progressing": 4,
            "time_progressing": 10,
            "pathway_complete": "training_pathway2_complete",
            "time_hold": 120,
        },
        "course6": {
            "drop_out_progressing": 0.05,
            "drop_out_hold": 0,
            "capacity_progressing": 4,
            "time_progressing": 6,
            "time_hold": 120,
        },
        "course7": {
            "drop_out_progressing": 0.0,
            "drop_out_hold": 0,
            "capacity_progressing": 8,
            "time_progressing": 2,
            "pathway_complete": "training_pathway3_complete",
            "time_hold": 120,
        },


    },
    "schedule": {
        "course1": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        "course2": [2, 3, 5, 6, 7, 8, 9, 10, 11],
        "course3": [1, 3, 4, 6, 7, 9, 11],
        "course4": [1, 4, 7, 10],
        "course5": [1, 4, 7, 10],
        "course6": [2, 3, 4, 6, 7, 9, 10],
        "course7": [1, 3, 4, 5, 6, 8, 9, 10, 11],


    },
}

In [5]:
dfs = []

for k, v in DEFAULT_PARAMETERS["schedule"].items():
    
    df = pd.DataFrame(index=range(1,37))
    df[f'{k}_start'] = ''
    count = 0
    for i in df.index:
        if i%12 == 0 and 12 in DEFAULT_PARAMETERS["schedule"][k]:
            df.loc[i,f'{k}_start'] = 1
        elif i%12 in DEFAULT_PARAMETERS["schedule"][k]:
            df.loc[i,f'{k}_start'] = 1
        else:
            df.loc[i,f'{k}_start'] = 0  
    df[f'{k}_cum'] = df.rolling(window=DEFAULT_PARAMETERS['pipeline'][k]['time_progressing'], min_periods=1).sum()
    for i in range(1,37):
        if df.loc[i,f'{k}_start'] == 1:
            df[f'{k}_{count}_left'] = ''
            df.loc[i, f'{k}_{count}_left'] = DEFAULT_PARAMETERS['pipeline'][k]['time_progressing']
            count += 1
            
    for i in range(1,37):
        for col in df.loc[:,df.columns.str.endswith('left')].columns:
            if df.loc[i,col]:
                if df.loc[i,col] > 1 and i+1 <= 36:
                    df.loc[i+1,col] = df.loc[i,col] -1
                    
    
    df = df.replace('',pd.NA)
    df = df.dropna(axis='columns', how='all')
    dfs.append(df)

In [6]:
final_df = pd.concat(dfs, axis=1)

final_df = final_df.tail(12).reset_index(drop=True)
final_df.index = range(1,13)

---

In [7]:
import networkx as nx
import enum
import math
from training_simulator.stages import INIT, PipelineStage
from training_simulator.model import PipelineModel
class Stage(enum.Enum):
    INIT = "init"
    COURSE1 = "course1"
    COURSE2 = "course2"
    COURSE3 = "course3"
    COURSE4 = "course4"
    COURSE5 = "course5"
    COURSE6 = "course6"
    COURSE7 = "course7"


In [8]:
def check_restream(model_params, career_pathway_file):
    df = pd.read_csv(career_pathway_file)
   
    return df

In [9]:
career_pathway_file="career_pathway.csv"
df = check_restream(DEFAULT_PARAMETERS, career_pathway_file)

cp = nx.from_pandas_edgelist(
        df,
        source="fromstage",
        target="tostage",
        edge_attr=True,
        create_using=nx.DiGraph())

In [10]:
def build_schedule(parameters: dict):
    return parameters["schedule"]

schedules = build_schedule(DEFAULT_PARAMETERS)

In [13]:
def build_stage_map(parameters: dict, cp: nx.DiGraph):
    stagemap = {}
    stagemap[Stage.INIT.value] = INIT(**(parameters["pipeline"][Stage.INIT.value]))
    for node in cp.nodes:
        # generate 5 digit int id
        unique_id = f"0"
        neighbors = cp.adj[node]
        stage = PipelineStage(
            **(parameters["pipeline"][node]),
            hold_progressing_trainees=[],
            stage_id=unique_id,
            stage_name=node,
            adjacent_stages=neighbors,
        )

        stagemap[node] = stage

    return stagemap

stage_map = build_stage_map(DEFAULT_PARAMETERS, cp)

In [18]:
def max_throughput_func(stage, stream_ratio = 1):
    if stage != 'init':
        return len(schedules[stage]) * stage_map[stage].capacity_progressing * stream_ratio
    else:
        return len(schedules[stage]) * DEFAULT_PARAMETERS['pipeline']['init']['new_trainees']

  
       
def find_minimum_course_throughput(stage: str) -> list:
    """calculate the upstream bottlenecks staing from any stage

    Args:
        params (dict): iteration param set
        stage (str): stage name

    Returns:
        list: list of upstream stages and thier own max thorughputs starting upstream from a specific stage
    """
    prev_stage_throughtput = []
    prev_stage_throughtput.append(max_throughput_func(stage))
    for course_pair in list(nx.edge_dfs(cp,stage, orientation='reverse')):
        prev_stage_throughtput.append(max_throughput_func(course_pair[0]))
    return min(prev_stage_throughtput)


        
avg_course_attendance = {}
for stage in cp.nodes:  
    print(stage)
    if stage != Stage.INIT.value:
        avg_course_attendance[stage] = round(find_minimum_course_throughput(stage=stage)/len(schedules[stage]), 2)
        print(f"Avg students per stage, {stage}, {avg_course_attendance[stage]: .02f}")

course1
Avg students per stage, course1,  21.00
course2
Avg students per stage, course2,  11.00
course3
Avg students per stage, course3,  10.00
course5
Avg students per stage, course5,  4.00
course6
Avg students per stage, course6,  4.00
course4
Avg students per stage, course4,  4.00
course7
Avg students per stage, course7,  3.11


In [19]:
def total_init_trainees(stage, month):
    return round(final_df.loc[month, stage +'_cum'] * avg_course_attendance[stage],2)

In [26]:
def split_trainees(stage, trainees, month):
    df = final_df.loc[:,final_df.columns.str.startswith(stage)]
    df = df.loc[month].to_frame().T.dropna(axis='columns', how='all')
    splits = df[f'{stage}_cum'].item()
    
    ans = []
    for i in range(-1,-1 * int(splits) - 1,-1):
        ans.append(f"{round(trainees/splits,0)} trainees with time progressing value of {DEFAULT_PARAMETERS['pipeline'][stage]['time_progressing'] - df.iloc[:,i].item(): .01f}")
    return ans

In [27]:
model_init_month = 5

for stage in cp.nodes:  
    if stage != Stage.INIT.value:
        print(f"int {stage} with {total_init_trainees(stage, model_init_month)}\n{split_trainees(stage, total_init_trainees(stage, model_init_month), model_init_month)} ")

int course1 with 42.0
['21.0 trainees with time progressing value of  0.0', '21.0 trainees with time progressing value of  1.0'] 
int course2 with 33.0
['11.0 trainees with time progressing value of  0.0', '11.0 trainees with time progressing value of  2.0', '11.0 trainees with time progressing value of  3.0'] 
int course3 with 10.0
['10.0 trainees with time progressing value of  1.0'] 
int course5 with 12.0
['4.0 trainees with time progressing value of  1.0', '4.0 trainees with time progressing value of  4.0', '4.0 trainees with time progressing value of  7.0'] 
int course6 with 12.0
['4.0 trainees with time progressing value of  1.0', '4.0 trainees with time progressing value of  2.0', '4.0 trainees with time progressing value of  3.0'] 
int course4 with 16.0
['4.0 trainees with time progressing value of  1.0', '4.0 trainees with time progressing value of  4.0', '4.0 trainees with time progressing value of  7.0', '4.0 trainees with time progressing value of  10.0'] 
int course7 with 

---

In [28]:
def split_trainees(stage, trainees, month):
    df = final_df.loc[:,final_df.columns.str.startswith(stage)]
    df = df.loc[month].to_frame().T.dropna(axis='columns', how='all')
    splits = df[f'{stage}_cum'].item()
    
    pilot_starts = []

    for i in range(-1,-1 * int(splits) - 1,-1):
        pilot_starts.append((round(trainees/splits,0), DEFAULT_PARAMETERS['pipeline'][stage]['time_progressing'] - df.iloc[:,i].item()))
    return pilot_starts

In [30]:
model_init_month = 5
warm_start = {}
for stage in cp.nodes:  
    if stage != Stage.INIT.value:
        warm_start[stage] = split_trainees(stage, total_init_trainees(stage, model_init_month), model_init_month)           

In [31]:
warm_start

{'course1': [(21.0, 0), (21.0, 1)],
 'course2': [(11.0, 0), (11.0, 2), (11.0, 3)],
 'course3': [(10.0, 1)],
 'course5': [(4.0, 1), (4.0, 4), (4.0, 7)],
 'course6': [(4.0, 1), (4.0, 2), (4.0, 3)],
 'course4': [(4.0, 1), (4.0, 4), (4.0, 7), (4.0, 10)],
 'course7': [(3.0, 0), (3.0, 1)]}

In [32]:
for stage in cp.nodes:  
    if stage != Stage.INIT.value:
        for i in warm_start[stage]:
            print(i[0], i[1])

21.0 0
21.0 1
11.0 0
11.0 2
11.0 3
10.0 1
4.0 1
4.0 4
4.0 7
4.0 1
4.0 2
4.0 3
4.0 1
4.0 4
4.0 7
4.0 10
3.0 0
3.0 1
